In [12]:
import sys
sys.path.append('..')
sys.path.append('../..')
import os
from utils import *
from student_utils_sp18 import *
from networkx.algorithms.traversal import *
from networkx.algorithms.approximation import *
from vasilis_solver import *
from simple_solver import *
from tqdm import tqdm
import matplotlib.pyplot as plt
import re
%matplotlib inline

In [21]:
def ensemble(G, start_index):
    
    # solver 1
    t1, c1 = simple(G, start_index)
    
    score1, text1 = cost_of_solution(G, t1, c1)
    total_cost1 = re.search('\d+(\.\d{1,100000})?', text1.split("\n")[2]).group(0)
    
    # solver 2
    t2, c2 = vasilis_solver(G, start_index)
    
    score2, text2 = cost_of_solution(G, t2, c2)
    total_cost2 = re.search('\d+(\.\d{1,100000})?', text2.split("\n")[2]).group(0)
         
    
    if total_cost1<total_cost2:
        return t1, c1
    else:
        return t2, c2

In [22]:
solve_all(ensemble)


  6%|▌         | 41/716 [00:50<13:45,  1.22s/it]

pipes


  9%|▉         | 66/716 [01:29<14:39,  1.35s/it]

KeyboardInterrupt: 

In [2]:
def solve_all(solver):
    indirectory = "inputs/"
    outdirectory = "outputs/"
    
    #os.makedirs(outdirectory)
    
    infiles = os.listdir(indirectory)
    
    solution_data = {}
    
    for input_file in tqdm(infiles):
            input_data = read_file(indirectory+input_file)
            number_of_kingdoms, list_of_kingdom_names, starting_kingdom, adjacency_matrix = data_parser(input_data)
            
            G, start_index = get_G(number_of_kingdoms, list_of_kingdom_names, starting_kingdom, adjacency_matrix)
            try: 
                closed_walk, conquered_kingdoms = solver(G, start_index)
            except KeyError as e:
                print(e)
                print("Error in input file: {}".format(input_file))
            
            
            # get the score of our solution
            score, text = cost_of_solution(G, closed_walk, conquered_kingdoms)
            
            try:
                conquering_cost = re.search('\d+(\.\d{1,100000})?', text.split("\n")[0]).group(0)
                travelling_cost = re.search('\d+(\.\d{1,100000})?', text.split("\n")[1]).group(0)
                total_cost = re.search('\d+(\.\d{1,100000})?', text.split("\n")[2]).group(0)
                
                solution_data[indirectory+input_file] = {"conquering_cost":conquering_cost, "travelling_cost":travelling_cost,
                                                    "total_cost":total_cost}
            except AttributeError:
                print(indirectory+input_file)
                print(text)
       
            closed_walk = [list_of_kingdom_names[i] for i in closed_walk]
            conquered_kingdoms = [list_of_kingdom_names[i] for i in conquered_kingdoms]
            

            output_file = outdirectory + input_file[:-3] +".out"
            write_data_to_file(output_file, closed_walk, ' ')
            write_to_file(output_file, '\n', append=True)
            write_data_to_file(output_file, conquered_kingdoms, ' ', append=True)
            
    return solution_data

In [6]:
def get_G(number_of_kingdoms, list_of_kingdom_names, starting_kingdom, adjacency_matrix):
    
    name2index = {}
    index2name = {}

    count = 0
    for name in list_of_kingdom_names:
        name2index[name] = count
        index2name[count] = name
        count += 1

    G = adjacency_matrix_to_graph(adjacency_matrix)
    
    color = {}

    for node in G.nodes():
            color[node] = "white"


    conquering_cost = {}
    for node in G.nodes():
        conquering_cost[node] = adjacency_matrix[node][node]

    nx.set_node_attributes(G, conquering_cost, "conquering_cost")
    nx.set_node_attributes(G, color, "color")
    
    return G, name2index[starting_kingdom]